# Сравнение результатов для глобального и локального выравнивания двух последовательностей

In [87]:
import numpy as np
a2 = "AATCGAAGTCA"
b2 = "AACGCAGAAT"
a = "taactttgcattc"
b = "ataaatatcactc"
match = 1
gap = -1
mismatch = -1
match2 = 5
gap2 = -7
mismatch2 = -3

In [78]:
def score_fun(x,y):
    if x == y:
        return match
    if x == "_" or y == "_":
        return gap
    return mismatch

In [79]:
def score_fun2(x,y):
    if x == y:
        return match2
    if x == "_" or y == "_":
        return gap2
    return mismatch2

Ниже представлена реализация алгоритма локального выравнивания:

In [83]:
def local_align(seq1, seq2, score = score_fun):
    M = np.zeros((len(seq2)+1, len(seq1) + 1))
    max_ = 0
    best_ij = (0,0)

    for i in range(1, len(seq2) + 1):
        for j in range(1, len(seq1) + 1):
           
            M[i][j] = max(M[i][j-1] + gap, M[i-1][j] + gap, M[i-1][j-1] + score(seq2[i-1], seq1[j-1]))
           
            if M[i][j] >= max_:
                max_ = M[i][j]
                best_ij = (i,j)

    al1 = ""
    al2 = ""
    j, i = best_ij
    while (i > 0 or j > 0) and M[j][i] > 0:
        curr = M[j][i]

        if i > 0 and j > 0 and seq1[i - 1] == seq2[j - 1]:
            al1 = seq1[i - 1] + al1
            al2 = seq2[j - 1] + al2
            i -= 1
            j -= 1
        elif i > 0 and (curr == M[j][i - 1] + mismatch or curr == M[j][i - 1] + gap) and M[j][i - 1] > 0:
            al1 = seq1[i - 1] + al1
            al2 = "_" + al2
            i -= 1
        else:
            al1 = "_" + al1
            al2 = seq2[j - 1] + al2
            j -= 1
 
    print(al1, al2, sep="\n")


Для сравнения будем использовать алгоритм Нидлмана-Вунша из предыдущего домашнего задания:

In [84]:
def needleman_wunsch(a, b, score = score_fun):
    n, m = len(a), len(b)
    match = 3
    gap = -2
    mismatch = -1
    mat = np.zeros((m + 1, n + 1))
    
    for i in range(0, m + 1):
            mat[i][0] = gap * i
    for j in range(0, n + 1):
            mat[0][j] = gap * j
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            top = mat[i-1][j] + gap
            left = mat[i][j-1] + gap
            diag = mat[i-1][j-1] + score(a[j-1],b[i-1])
            mat[i][j] = max(diag, left , top)      
    seq1 = ""
    seq2 = ""

    i = m
    j = n
    while i > 0 and j > 0:
        score_curr = mat[i][j]
        score_top = mat[i][j-1]
        score_left = mat[i-1][j]
        score_diag = mat[i-1][j-1]
        if score_curr == score_diag + score(a[j-1], b[i-1]):
            seq1 = a[j-1] + seq1
            seq2 = b[i-1] + seq2
            i -= 1
            j -= 1
        elif score_curr == score_top + gap:
            seq1 = a[j-1] + seq1
            seq2 = '_' + seq2
            j -= 1
        elif score_curr == score_left + gap:
            seq1 = '_' + seq1
            seq2 = b[i-1] + seq2
            i -= 1
    while j > 0:
        seq1 = a[j-1] + seq1
        seq2 = '_' + seq2
        j -= 1
    while i > 0:
        seq1 = '_' + seq1
        seq2 = b[i-1] + seq2
        i -= 1


    print(seq1, seq2, sep = "\n")

In [85]:
needleman_wunsch(a,b)
print("\n")
local_align(a,b)

_taactttgcattc
ataaatat_cactc


taac_tt_tgcat_tc
taa_a_tat_ca_ctc


In [86]:
needleman_wunsch(a, b, score = score_fun2)
print("\n")
local_align(a, b, score = score_fun2)

_taactttgcattc
ataaatat_cactc


t_aact_ttgca_ttc
taaa_ta_t_cac_tc
